In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split
import torchvision
from torchvision.datasets import CIFAR10
from torchvision.transforms import ToTensor, Grayscale, Normalize, Compose
from torchvision.utils import make_grid
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# Define transformation for preprocessing
transform = Compose([
    Grayscale(num_output_channels=1),  # Convert to grayscale
    ToTensor(),  # Convert image to PyTorch tensor
    Normalize((0.5,), (0.5,))  # Normalize to range [-1, 1]
])

In [4]:
# Load CIFAR-10 dataset
train_dataset = CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = CIFAR10(root='./data', train=False, download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [5]:
batch_size = 256

In [6]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [7]:
len(train_loader), len(test_loader)

(196, 40)

## QNN

In [23]:
pip install pennylane

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
import pennylane as qml
from pennylane import numpy as np

In [38]:
train_dataset[0][0].shape

torch.Size([1, 32, 32])

In [ ]:
class QuanvolutionalLayer(nn.Module):
    def __init__(self, filter_size=2, n_layers=1, stride=2, padding=0):
        """
        Quanvolutional Layer in PyTorch
        - filter_size: Size of the quantum filter (e.g., 2x2)
        - n_layers: Number of random quantum layers
        - stride: Stride for the filter movement
        - padding: Padding around the image (not implemented)
        """
        super(QuanvolutionalLayer, self).__init__()
        self.filter_size = filter_size
        self.n_layers = n_layers
        self.stride = stride
        self.padding = padding

        # Define a quantum device with the required number of qubits (equal to filter_size^2)
        self.dev = qml.device("default.qubit", wires=filter_size**2)

        # Generate random quantum circuit parameters
        self.rand_params = np.random.uniform(0, 2*np.pi, size=(n_layers, filter_size**2))

    def quantum_circuit(self, inputs):
        """
        Quantum circuit for the quanvolutional layer.
        This function encodes the inputs into quantum states, applies quantum gates,
        and returns expectation values of Pauli-Z measurements.
        """
        @qml.qnode(self.dev, interface="torch")
        def circuit(inputs):
            # Encode classical inputs as quantum rotations
            for i in range(self.filter_size**2):
                qml.RY(inputs[i] * np.pi, wires=i)

            # Apply random quantum layers
            qml.templates.RandomLayers(self.rand_params, wires=list(range(self.filter_size**2)))

            # Measure expectation values of Pauli-Z
            return [qml.expval(qml.PauliZ(i)) for i in range(self.filter_size**2)]

        return torch.tensor(circuit(inputs), dtype=torch.float32)  # Convert list to Tensor

    def forward(self, x):
        """
        Applies the quantum transformation to the input image.
        - x: Input image of shape [batch_size, 1, H, W]
        Returns a transformed feature map.
        """
        
        batch_size, channels, height, width = x.shape
        
        if self.padding != 0:
            # Apply padding to input image
            x = torch.nn.functional.pad(x, (self.padding, self.padding, self.padding, self.padding), mode='constant', value=0)

        # Compute output size using the correct formula
        output_size = ((height + 2 * self.padding - self.filter_size) // self.stride) + 1
        print(f"generated output size is : {output_size}")
        # Create an empty tensor to store the output
        output = torch.zeros(batch_size, self.filter_size**2, output_size, output_size)

        # Apply quantum filters to patches
        for i in range(output_size):
            for j in range(output_size):
                row_start = i * self.stride
                col_start = j * self.stride
                patch = x[:, :, row_start:row_start+self.filter_size, col_start:col_start+self.filter_size]
                patch = patch.contiguous().reshape(batch_size, -1)  # Ensure contiguous memory

                # Apply quantum circuit on each patch and store results
                q_results = torch.stack([self.quantum_circuit(p) for p in patch])

                output[:, :, i, j] = q_results  # Store at corrected index

        return output


In [39]:
# this code is used for just test the working of above function 

# Create a dummy batch of images [batch_size, channels, height, width]
dummy_images = torch.rand(4, 1, 32, 32)  # 4 grayscale images of size 28 X 28

# Instantiate the quanvolutional layer
quanv = QuanvolutionalLayer(filter_size=3, n_layers=2, stride=1, padding=1)

# Apply the quanvolutional transformation
output = quanv(dummy_images)

# Print the shape of the output
print("Input shape:", dummy_images.shape)    # Expected: [4, 1, 32, 32]
print("Output shape:", output.shape)         # Expected: [4, 4, 16, 16] (since 2x2 filter → 4 output channels)


generated output size is : 32


c:\Users\dhruv\Python_Envs\dl_env\lib\site-packages\pennylane\math\interface_utils.py:127: UserWarning: Contains tensors of types {'autograd', 'torch'}; dispatch will prioritize TensorFlow, PyTorch, and Jax over Autograd. Consider replacing Autograd with vanilla NumPy.
  warnings.warn(


Input shape: torch.Size([4, 1, 32, 32])
Output shape: torch.Size([4, 9, 32, 32])


In [41]:
# this code is used for just test the working of above function 

# Create a dummy batch of images [batch_size, channels, height, width]
dummy_images = torch.rand(4, 1, 32, 32)  # 4 grayscale images of size 28 X 28

# Instantiate the quanvolutional layer
quanv = QuanvolutionalLayer(filter_size=3, n_layers=1, stride=2, padding=1)

# Apply the quanvolutional transformation
output = quanv(dummy_images)

# Print the shape of the output
print("Input shape:", dummy_images.shape)    # Expected: [4, 1, 32, 32]
print("Output shape:", output.shape)         # Expected: [4, 4, 16, 16] (since 2x2 filter → 4 output channels)


generated output size is : 16


c:\Users\dhruv\Python_Envs\dl_env\lib\site-packages\pennylane\math\interface_utils.py:127: UserWarning: Contains tensors of types {'autograd', 'torch'}; dispatch will prioritize TensorFlow, PyTorch, and Jax over Autograd. Consider replacing Autograd with vanilla NumPy.
  warnings.warn(


Input shape: torch.Size([4, 1, 32, 32])
Output shape: torch.Size([4, 9, 16, 16])


In [ ]:
## this code is used for just test the working of above function 

# # Create a dummy batch of images [batch_size, channels, height, width]
# dummy_images = torch.rand(4, 1, 32, 32)  # 4 grayscale images of size 32x32

# # Instantiate the quanvolutional layer
# quanv = QuanvolutionalLayer(filter_size=2, n_layers=1, stride=2)

# # Apply the quanvolutional transformation
# output = quanv(dummy_images)

# # Print the shape of the output
# print("Input shape:", dummy_images.shape)    # Expected: [4, 1, 32, 32]
# print("Output shape:", output.shape)         # Expected: [4, 4, 16, 16] (since 2x2 filter → 4 output channels)


Input shape: torch.Size([4, 1, 32, 32])
Output shape: torch.Size([4, 4, 16, 16])


In [ ]:
class Quanvolutional_Convolutional_NeuralNetwork(nn.Module):
    def __init__(self):
        super(Quanvolutional_Convolutional_NeuralNetwork, self).__init__()

        self.quanv1 = QuanvolutionalLayer(filter_size=3, n_layers=1, stride=1, padding=1)
        self.conv1 = nn.Conv2d(in_channels=9, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=2, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(32 * 8 * 8, 128)  # Output size after pooling
        self.fc2 = nn.Linear(128, 10)  # Final classification (CIFAR-10 has 10 classes)

        # # Fully connected layer for classification
        # self.fc = nn.Linear(4 * 16 * 16, 10)  # CIFAR-10 has 10 classes


        # # 1️⃣ Quanvolutional Layer (Extract quantum features)
        # self.quanv = QuanvolutionalLayer(filter_size=2, n_layers=1, stride=2)

        # 2️⃣ Convolutional Layers (Classic CNN model)
        # self.conv1 = nn.Conv2d(in_channels=4, out_channels=16, kernel_size=3, stride=1, padding=1)
        # self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)

        # 3️⃣ Pooling Layer
        # self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # 4️⃣ Fully Connected Layers (Dense Layers)
        # self.fc1 = nn.Linear(32 * 8 * 8, 128)  # Output size after pooling
        # self.fc2 = nn.Linear(128, 10)  # Final classification (CIFAR-10 has 10 classes)



    def forward(self, x):
        # Apply quanvolutional layer
        x = self.quanv1(x)

        # 2️⃣ Pass through CNN layers
        x = self.conv1(x)
        x = F.relu()  # Conv layer 1 + ReLU
        x = self.pool(x)
        x = self.conv2(x)           # Apply max pooling
        x = F.relu(x)  # Conv layer 2 + ReLU
        x = self.pool(x)           # Apply max pooling again

        # 3️⃣ Flatten output
        x = x.view(x.shape[0], -1)  

        # 4️⃣ Fully connected layers
        x = self.fc1(x)
        x = F.relu(x)  
        x = self.fc2(x)  # No activation here, since we'll apply softmax in loss function

        return x  # Raw 

        # # Flatten feature maps
        # x = x.view(x.shape[0], -1)  

        # # Apply fully connected layer
        # x = self.fc(x)

        # return F.log_softmax(x, dim=1)  # Use log-softmax for numerical stability


In [38]:
# Create a dummy batch of grayscale CIFAR-10 images
dummy_images = torch.rand(4, 1, 32, 32)  # Batch size: 4, Channels: 1, Height: 32, Width: 32

# Instantiate the model
model = QuanvolutionalNeuralNetwork()

# Forward pass
output = model(dummy_images)

# Print the output shape
print("Model output shape:", output.shape)  # Expected: [4, 10] (batch_size, num_classes)


Model output shape: torch.Size([4, 10])


In [39]:
import torch.optim as optim

In [ ]:
# Define loss function and optimizer
criterion = nn.NLLLoss()  # Negative Log Likelihood Loss (for log-softmax output)
optimizer = optim.SGD(model.parameters(), lr=0.05, momentum=0.9)

In [41]:
def train(model, train_loader, criterion, optimizer, epochs=30):
    model.train()  # Set model to training mode

    for epoch in range(epochs):
        total_loss = 0
        correct = 0
        total = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)  # Move to GPU if available

            optimizer.zero_grad()  # Reset gradients

            outputs = model(images)  # Forward pass
            loss = criterion(outputs, labels)  # Compute loss
            loss.backward()  # Backpropagation
            optimizer.step()  # Update weights

            total_loss += loss.item()
            preds = outputs.argmax(dim=1)  # Get class with highest probability
            correct += (preds == labels).sum().item()
            total += labels.size(0)

        # Print epoch stats
        print(f"Epoch [{epoch+1}/{epochs}] | Loss: {total_loss/len(train_loader):.4f} | Accuracy: {correct/total:.4f}")

    print("Training complete!")


In [42]:
def test(model, test_loader, criterion):
    model.eval()  # Set model to evaluation mode
    test_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():  # No gradient calculation needed
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            test_loss += criterion(outputs, labels).item()

            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    print(f"Test Loss: {test_loss/len(test_loader):.4f} | Test Accuracy: {correct/total:.4f}")


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Use GPU if available
model.to(device)  # Move model to GPU

# Train the model
train(model, train_loader, criterion, optimizer, epochs=30)

# Evaluate the model
test(model, test_loader, criterion)
